In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"
import json
import numpy as np
from typing import Any, Dict, List, Union
from tensorflow.keras.preprocessing import image


In [2]:
# Facenet512
from facenetlib.Facenet_standalone import InceptionResNetV1
# RetinaFace
from retina import detect_faces, create_retinaface_model
# Preprocessing utils
from facenetlib import preprocessing, image_utils

In [3]:
# =========================
# Load Models
# =========================
print("[INFO] Loading Facenet512...")
embedder = InceptionResNetV1(dimension=512)
embedder.load_weights("facenetlib/facenet512_weights.h5")

print("[INFO] Loading RetinaFace...")
detector = create_retinaface_model()


[INFO] Loading Facenet512...


[INFO] Loading RetinaFace...
Building RetinaFace model architecture...
Loading weights into model...
Weights loaded successfully!


In [8]:
# =========================
# Batch Processing Function
# =========================
def batch_represent(
    img_paths: List[str],
    threshold: float = 0.9,
    normalization: str = "base"
) -> Dict[str, List[Dict[str, Any]]]:
    """
    Deteksi semua wajah dari setiap gambar, proses embedding dalam batch,
    dan return hasil per file.
    """
    all_faces = []   # semua crop wajah (siap masuk model)
    meta = []        # mapping embedding -> filename + info wajah

    # 1. Baca semua gambar
    imgs_rgb = []
    imgs_bgr = []
    for p in img_paths:
        img_rgb, _ = image_utils.load_image(p)
        imgs_rgb.append(img_rgb)
        imgs_bgr.append(img_rgb[:, :, ::-1])  # BGR untuk detektor

    # 2. Deteksi wajah & kumpulkan semua crop
    for idx, (img_path, img_bgr, img_rgb) in enumerate(zip(img_paths, imgs_bgr, imgs_rgb)):
        detections = detect_faces(
            img_path=img_bgr,
            threshold=threshold,
            model=detector
        )

        # Pastikan format list of dict
        if isinstance(detections, dict) and all(isinstance(v, dict) for v in detections.values()):
            detections = list(detections.values())

        for det in detections:
            coords = det["facial_area"]
            if isinstance(coords, list):
                x1, y1, x2, y2 = coords
            else:  # dict
                x1, y1 = coords["x"], coords["y"]
                x2, y2 = x1 + coords["w"], y1 + coords["h"]

            face_conf = det.get("confidence", det.get("score", 0.0))
            crop = img_rgb[int(y1):int(y2), int(x1):int(x2)]

            # Resize & normalize
            crop_resized = preprocessing.resize_image(crop, target_size=(160, 160))
            crop_norm = preprocessing.normalize_input(crop_resized, normalization=normalization)

            if crop_norm.ndim == 4 and crop_norm.shape[0] == 1:
                crop_norm = crop_norm[0]

            all_faces.append(crop_norm)
            meta.append({
                "filename": os.path.basename(img_path),
                "facial_area": {
                    "x": int(x1), "y": int(y1),
                    "w": int(x2 - x1), "h": int(y2 - y1)
                },
                "confidence": float(face_conf)
            })

    if not all_faces:
        return {}

    # 3. Convert ke batch array
    batch_faces = np.array(all_faces)
    print(f"[INFO] Processing {len(batch_faces)} faces in batch...")

    # 4. Sekali forward pass ke Facenet
    embeddings = embedder.predict(batch_faces, verbose=0)

    # 5. Kelompokkan hasil per file
    results_by_file = {}
    for m, emb in zip(meta, embeddings):
        m["embedding"] = emb.astype(float).tolist()
        results_by_file.setdefault(m["filename"], []).append(m)

    return results_by_file


In [9]:
# =========================
# Contoh penggunaan
# =========================
if __name__ == "__main__":
    input_folder = "images"
    output_json = "embedding/embeddings_batch.json"

    img_files = [
        os.path.join(input_folder, f)
        for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    results = batch_represent(img_files)

    with open(output_json, "w") as f:
        json.dump(results, f, indent=2)

    print(f"✅ Batch embeddings saved to {output_json}")

[INFO] Processing 9 faces in batch...
✅ Batch embeddings saved to embedding/embeddings_batch.json
